In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('default')

from z3 import *
import numpy as np

import sys
sys.path.append("../common")

from utils import visualize, read_instance, PositionedRectangle

#reloads external modules when they are changed
%load_ext autoreload
%autoreload 2

In [ ]:
def order_encode(value, vmin = 0, vmax = -1):
    """
    Parameters
    ----------
    value : int
        value to encode
    vmin : int, defualt 0
        starting value of the domain
    vmax : int, optional
        final value of the domain. Defaults to value

    Returns
    -------
    list of bool
        the order encoded value in the form [False, ..., False, True, True, ...]
    """
    if vmax < value:
        if vmax != -1:
            print("Attention! The given length of the encoding is smaller than the value to encode!")
        vmax = value
    
    if value < vmin:
        print("Attention! The encoded value is smaller than the minimum value of the domain!")


    encoding = []
    for i in range(vmin, vmax):
        encoding.append(value <= i)
    return encoding


def order_decode(encoding, vmin = 0):
    """
    Parameters
    ----------
    encoding : list of bool
        order encoded value in the form [False, ..., False, True, True, ...]
    vmin : int, default 0
        starting value of the domain of the encoded value

    Returns
    -------
    int
        the decoded value
    """
    for i, val in enumerate(encoding):
        if val:
            return i + vmin
    return len(encoding) + vmin

In [ ]:
ins_filename = "ins-15.txt"
W, n, rectangles = read_instance("../instances/" + ins_filename)

max_height = max([r.h for r in rectangles])
sum_height = sum([r.h for r in rectangles])
#H = (max_height + sum_height) // 2

total_area = sum([r.h*r.w for r in rectangles])
H = int(np.ceil((total_area)/W))

print(total_area)

px = [[Bool(f"px_{i+1}_{e}") for e in range(W - rectangles[i].w)] for i in range(n)]
py = [[Bool(f"py_{i+1}_{f}") for f in range(H - rectangles[i].h)] for i in range(n)]

lr = [[Bool(f"lr_{i+1}_{j+1}") for j in range(n)] for i in range(n)]
ud = [[Bool(f"ud_{i+1}_{j+1}") for j in range(n)] for i in range(n)]

In [ ]:
s = Solver()

# check validity
for i in range(n):
    if rectangles[i].w > W:
        s.add(False)
        print("Invalid: there is a rectangle that is wider than the strip!")
        break

    if rectangles[i].h > H:
        s.add(False)
        print("Invalid: there is a rectangle that is taller than the strip!")
        break


# ordering constraints
for i in range(n):
    for e in range(len(px[i]) - 1):
        s.add(Implies(px[i][e], px[i][e + 1]))

for i in range(n):
    for f in range(len(py[i]) - 1):
        s.add(Implies(py[i][f], py[i][f + 1]))


# non overlapping constraints (4-literal clauses)
for i in range(n):
    for j in range(i):
        to_add = []
        if rectangles[i].w + rectangles[j].w <= W:
            to_add += [lr[i][j], lr[j][i]]
        if rectangles[i].h + rectangles[j].h <= H:
            to_add += [ud[i][j], ud[j][i]]

        s.add(Or(to_add))


# non overlapping constraints (3-literal clauses)
for i in range(n):
    for j in range(n):
        if i==j: continue

        # rectangle j cannot be at the left of the right edge of rectangle i
        for k in range(min(rectangles[i].w, len(px[j]))):
            s.add(Or(Not(lr[i][j]), Not(px[j][k])))

        if rectangles[i].w + rectangles[j].w <= W:
            for e in range(W):
                e1 = e + rectangles[i].w

                if e < len(px[i]) and e1 < len(px[j]):
                    s.add(Or(Not(lr[i][j]), px[i][e], Not(px[j][e1])))

                if e < len(px[i]) and e1 >= len(px[j]):
                    s.add(Or(Not(lr[i][j]), px[i][e]))

                if e >= len(px[i]) and e1 < len(px[j]):
                    s.add(Or(Not(lr[i][j]), Not(px[j][e1])))


        # rectangle j cannot be under of the top edge of rectangle i
        for k in range(min(rectangles[i].h, len(py[j]))):
            s.add(Or(Not(ud[i][j]), Not(py[j][k])))

        if rectangles[i].h + rectangles[j].h <= H:
            for f in range(H):
                f1 = f + rectangles[i].h

                if f < len(py[i]) and f1 < len(py[j]):
                    s.add(Or(Not(ud[i][j]), py[i][f], Not(py[j][f1])))

                if f < len(py[i]) and f1 >= len(py[j]):
                    s.add(Or(Not(ud[i][j]), py[i][f]))

                if f >= len(py[i]) and f1 < len(py[j]):
                    s.add(Or(Not(ud[i][j]), Not(py[j][f1])))

#print(s)

if s.check() == sat:
    m = s.model()
else:
    print("Failed to solve")

In [ ]:
px_eval = [[m.evaluate(px[i][j], model_completion = True) for j in range(len(px[i]))] for i in range(len(px))]
x_values = [order_decode(p) for p in px_eval]

py_eval = [[m.evaluate(py[i][j], model_completion = True) for j in range(len(py[i]))] for i in range(len(px))]
y_values = [order_decode(p) for p in py_eval]


positionedRectangles = [PositionedRectangle(x, y, rect.w, rect.h) for x, y, rect in zip(x_values, y_values, rectangles)]

fig, ax = plt.subplots(figsize = (8, 8), dpi = 100)

fig, ax = visualize(W, H, positionedRectangles, ax = ax)
fig.suptitle(f"{ins_filename} ({n} rectangles)")
fig.tight_layout(pad = 1)
#plt.savefig("../" + ins_filename.split('.')[0])
plt.show()